<a href="https://colab.research.google.com/github/GTmycology/dreambooth-colab/blob/main/Lora_Trainer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ⭐ Lora Trainer by Hollowstrawberry

This is based on the work of [Kohya-ss](https://github.com/kohya-ss/sd-scripts) and [Linaqruf](https://colab.research.google.com/github/Linaqruf/kohya-trainer/blob/main/kohya-LoRA-dreambooth.ipynb). Thank you!


### ⭕ Disclaimer
The purpose of this document is to research bleeding-edge technologies in the field of machine learning.  
Please read and follow the [Google Colab guidelines](https://research.google.com/colaboratory/faq.html) and its [Terms of Service](https://research.google.com/colaboratory/tos_v3.html).

| |GitHub|🇬🇧 English|🇪🇸 Spanish|
|:--|:-:|:-:|:-:|
| 🏠 **Homepage** | [![GitHub](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/github.svg)](https://github.com/hollowstrawberry/kohya-colab) | | |
| 📊 **Dataset Maker** | [![GitHub](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/github.svg)](https://github.com/hollowstrawberry/kohya-colab/blob/main/Dataset_Maker.ipynb) | [![Open in Colab](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/colab-badge.svg)](https://colab.research.google.com/github/hollowstrawberry/kohya-colab/blob/main/Dataset_Maker.ipynb) | [![Abrir en Colab](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/colab-badge-spanish.svg)](https://colab.research.google.com/github/hollowstrawberry/kohya-colab/blob/main/Spanish_Dataset_Maker.ipynb) |
| ⭐ **Lora Trainer** | [![GitHub](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/github.svg)](https://github.com/hollowstrawberry/kohya-colab/blob/main/Lora_Trainer.ipynb) | [![Open in Colab](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/colab-badge.svg)](https://colab.research.google.com/github/hollowstrawberry/kohya-colab/blob/main/Lora_Trainer.ipynb) | [![Abrir en Colab](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/colab-badge-spanish.svg)](https://colab.research.google.com/github/hollowstrawberry/kohya-colab/blob/main/Spanish_Lora_Trainer.ipynb) |
| 🌟 **XL Lora Trainer** | [![GitHub](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/github.svg)](https://github.com/hollowstrawberry/kohya-colab/blob/main/Lora_Trainer_XL.ipynb) | [![Open in Colab](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/colab-badge.svg)](https://colab.research.google.com/github/hollowstrawberry/kohya-colab/blob/main/Lora_Trainer_XL.ipynb) |  |
| 🌟 **Legacy XL Trainer** | [![GitHub](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/github.svg)](https://github.com/hollowstrawberry/kohya-colab/blob/main/Lora_Trainer_XL_Legacy.ipynb) | [![Open in Colab](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/colab-badge.svg)](https://colab.research.google.com/github/hollowstrawberry/kohya-colab/blob/main/Lora_Trainer_XL_Legacy.ipynb) |  |

In [6]:
from accelerate.utils import write_basic_config
import os

repo_dir = "/content/kohya-trainer"
accelerate_config_file = os.path.join(repo_dir, "accelerate_config/config.yaml")

os.makedirs(os.path.dirname(accelerate_config_file), exist_ok=True)
write_basic_config(save_location=accelerate_config_file)

print(f"✅ Accelerate config created at: {accelerate_config_file}")


✅ Accelerate config created at: /content/kohya-trainer/accelerate_config/config.yaml


In [7]:
# @title
import os, re, toml
from time import time
from IPython.display import Markdown, display
from accelerate.utils import write_basic_config

# --- Core setup ---
COLAB = True
XFORMERS = True
SOURCE = "https://github.com/uYouUs/sd-scripts"
COMMIT = None
BETTER_EPOCH_NAMES = True
LOAD_TRUNCATED_IMAGES = True

# --- Project info ---
project_name = "playful_origins_v4_4"
model_url = "/content/drive/MyDrive/Loras/playful_origins_v4/sd-nsfw-v1-5-realism.ckpt"
images_folder = "/content/drive/MyDrive/Loras/playful_origins_v4/dataset_v2/class1/PlayfulOrigins_00001"
output_folder = "/content/drive/MyDrive/Loras/playful_origins_v4/output_lora"
config_folder = "/content/drive/MyDrive/Loras/playful_origins_v4"
log_folder = "/content/drive/MyDrive/Loras/playful_origins_v4/_logs"

config_file = os.path.join(config_folder, "training_config.toml")
dataset_config_file = os.path.join(config_folder, "dataset_config.toml")
repo_dir = "/content/kohya-trainer"
accelerate_config_file = os.path.join(repo_dir, "accelerate_config/config.yaml")

# --- Create accelerate config if missing ---
os.makedirs(os.path.dirname(accelerate_config_file), exist_ok=True)
if not os.path.exists(accelerate_config_file):
    write_basic_config(save_location=accelerate_config_file)
    print(f"✅ Accelerate config created at {accelerate_config_file}")
else:
    print(f"✅ Accelerate config already exists")

# --- Training parameters ---
resolution = 768
num_repeats = 10
preferred_unit = "Steps"
how_many = 2222
max_train_epochs = how_many if preferred_unit == "Epochs" else None
max_train_steps = how_many if preferred_unit == "Steps" else None
train_batch_size = 2
unet_lr = 5e-4
text_encoder_lr = 1e-4
lr_scheduler = "cosine_with_restarts"
lr_scheduler_number = 3
lr_scheduler_num_cycles = 3
lr_warmup_ratio = 0.05
min_snr_gamma_value = 5.0
lora_type = "LoRA"
network_dim = 32
network_alpha = 16
conv_dim = 8
conv_alpha = 4

# --- Dataset check ---
def validate_dataset():
    supported_types = (".png", ".jpg", ".jpeg", ".webp", ".bmp")
    print("\n💿 Checking dataset...")
    if not os.path.exists(images_folder):
        print(f"💥 Error: Dataset folder not found -> {images_folder}")
        return False
    imgs = [f for f in os.listdir(images_folder) if f.lower().endswith(supported_types)]
    txts = [f for f in os.listdir(images_folder) if f.lower().endswith(".txt")]
    print(f"📁 {images_folder}")
    print(f"📈 Found {len(imgs)} images with {num_repeats} repeats, equaling {len(imgs)*num_repeats} steps.")
    print(f"📝 Captions found: {len(txts)}")
    if len(imgs) == 0:
        print("💥 Error: No images found.")
        return False
    if len(txts) == 0:
        print("⚠️ WARNING: 0 captions found — check dataset!")
    return True

# --- Config creation ---
def create_config():
    dataset_config = {
        "general": {
            "resolution": resolution,
            "flip_aug": False,
            "caption_extension": ".txt",
            "enable_bucket": True,
            "bucket_reso_steps": 64,
            "bucket_no_upscale": False,
        },
        "datasets": [
            {
                "subsets": [
                    {
                        "num_repeats": num_repeats,
                        "image_dir": images_folder,
                        "class_tokens": None
                    }
                ]
            }
        ]
    }

    train_config = {
        "additional_network_arguments": {
            "unet_lr": unet_lr,
            "text_encoder_lr": text_encoder_lr,
            "network_dim": network_dim,
            "network_alpha": network_alpha,
            "network_module": "networks.lora",
        },
        "optimizer_arguments": {
            "learning_rate": unet_lr,
            "lr_scheduler": lr_scheduler,
            "lr_scheduler_num_cycles": lr_scheduler_num_cycles,
            "lr_warmup_steps": int(how_many * lr_warmup_ratio),
            "optimizer_type": "AdamW8bit",
        },
        "training_arguments": {
            "max_train_steps": max_train_steps,
            "train_batch_size": train_batch_size,
            "min_snr_gamma": min_snr_gamma_value,
            "mixed_precision": "fp16",
            "save_every_n_epochs": 1,
            "save_last_n_epochs": 14,
            "output_dir": output_folder,
            "logging_dir": log_folder,
            "output_name": project_name,
        },
        "model_arguments": {
            "pretrained_model_name_or_path": model_url,
            "v2": False,
        },
        "saving_arguments": {
            "save_model_as": "safetensors",
        },
    }

    os.makedirs(config_folder, exist_ok=True)
    with open(config_file, "w") as f:
        f.write(toml.dumps(train_config))
    with open(dataset_config_file, "w") as f:
        f.write(toml.dumps(dataset_config))

    print(f"📄 Configs saved to {config_folder}")

# --- Main ---
def main():
    print("\n🚀 Starting Playful Origins v4.4 Angel Run")
    print("\n✅ Dependencies already installed.")
    if not validate_dataset():
        return
    print(f"✅ Using existing model: {model_url}")
    create_config()

    print("\n⭐ Starting trainer...\n")
    os.chdir(repo_dir)
    os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
    os.environ["BITSANDBYTES_NOWELCOME"] = "1"
    os.environ["SAFETENSORS_FAST_GPU"] = "1"
    os.system(f"accelerate launch --config_file={accelerate_config_file} "
              f"--num_cpu_threads_per_process=1 train_network_wrapper.py "
              f"--dataset_config={dataset_config_file} --config_file={config_file}")

main()


✅ Accelerate config already exists

🚀 Starting Playful Origins v4.4 Angel Run

✅ Dependencies already installed.

💿 Checking dataset...
📁 /content/drive/MyDrive/Loras/playful_origins_v4/dataset_v2/class1/PlayfulOrigins_00001
📈 Found 30 images with 10 repeats, equaling 300 steps.
📝 Captions found: 30
✅ Using existing model: /content/drive/MyDrive/Loras/playful_origins_v4/sd-nsfw-v1-5-realism.ckpt
📄 Configs saved to /content/drive/MyDrive/Loras/playful_origins_v4

⭐ Starting trainer...



In [9]:
# 🔧 Reduce memory load for T4
import os

# Make PyTorch use expandable memory segments (helps fragmentation)
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# These are the lighter settings for your rerun:
train_batch_size = 1          # was 2
resolution = 704              # was 768
network_dim = 16              # was 32
network_alpha = 8             # half of dim

print("✅ Memory-safe config applied (batch=1, res=704, dim=16, alpha=8)")


✅ Memory-safe config applied (batch=1, res=704, dim=16, alpha=8)


In [11]:
# 🚀 Clean Launch — Low VRAM Safe Mode for T4 (Playful Origins v4.4)
import os, toml, time, re

print("🔄 Forcing clean launch in low-VRAM mode...")

# --- 🔧 Memory environment tweaks
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True,max_split_size_mb:256"
os.environ["TORCH_CUDA_ALLOC_CONF"] = "garbage_collection_threshold:0.8,max_split_size_mb:128"

# --- ✅ Safe training parameters
train_batch_size = 1
resolution = 640
network_dim = 8
network_alpha = 4
print(f"✅ Using resolution={resolution}, batch={train_batch_size}, dim={network_dim}, alpha={network_alpha}")

# --- 🗂️ Core paths
project_root = "/content/drive/MyDrive/Loras/playful_origins_v4"
dataset_path = f"{project_root}/dataset_v2/class1/PlayfulOrigins_00001"
model_ckpt = f"{project_root}/sd-nsfw-v1-5-realism.ckpt"
config_file = f"{project_root}/training_config.toml"
dataset_config_file = f"{project_root}/dataset_config.toml"
accelerate_config = "/content/kohya-trainer/accelerate_config/config.yaml"

# --- 🔁 Overwrite dataset config with new resolution & batch size
dataset_conf = {
    "general": {
        "resolution": resolution,
        "shuffle_caption": False,
        "keep_tokens": 0,
        "flip_aug": False,
        "caption_extension": ".txt",
        "enable_bucket": True,
        "bucket_reso_steps": 64,
        "bucket_no_upscale": False,
        "min_bucket_reso": 256,
        "max_bucket_reso": 1024,
    },
    "datasets": [
        {"subsets": [
            {"num_repeats": 10, "image_dir": dataset_path, "class_tokens": None}
        ]}
    ]
}
with open(dataset_config_file, "w") as f:
    toml.dump(dataset_conf, f)
print(f"📄 Updated dataset config: {dataset_config_file}")

# --- 🔁 Overwrite training config to ensure dim/alpha/batch apply
train_conf = {
    "additional_network_arguments": {
        "unet_lr": 5e-4,
        "text_encoder_lr": 1e-4,
        "network_dim": network_dim,
        "network_alpha": network_alpha,
        "network_module": "networks.lora",
        "network_train_unet_only": False,
    },
    "optimizer_arguments": {
        "learning_rate": 5e-4,
        "lr_scheduler": "cosine_with_restarts",
        "lr_scheduler_num_cycles": 3,
        "lr_warmup_steps": 100,
        "optimizer_type": "AdamW8bit",
    },
    "training_arguments": {
        "max_train_steps": 2222,
        "train_batch_size": train_batch_size,
        "save_every_n_epochs": 1,
        "save_last_n_epochs": 14,
        "clip_skip": 2,
        "min_snr_gamma": 5.0,
        "xformers": True,
        "lowram": True,
        "save_precision": "fp16",
        "mixed_precision": "fp16",
        "output_dir": f"{project_root}/output_lora",
        "logging_dir": f"{project_root}/_logs",
        "output_name": "playful_origins_v4_4",
        "log_prefix": "playful_origins_v4_4",
    },
    "model_arguments": {"pretrained_model_name_or_path": model_ckpt},
    "saving_arguments": {"save_model_as": "safetensors"},
}
with open(config_file, "w") as f:
    toml.dump(train_conf, f)
print(f"📄 Updated training config: {config_file}")

# --- 🌟 Launch trainer
os.chdir("/content/kohya-trainer")
print("\n⭐ Starting trainer...\n")
!accelerate launch --config_file={accelerate_config} --num_cpu_threads_per_process=1 train_network_wrapper.py \
  --dataset_config={dataset_config_file} --config_file={config_file}


🔄 Forcing clean launch in low-VRAM mode...
✅ Using resolution=640, batch=1, dim=8, alpha=4
📄 Updated dataset config: /content/drive/MyDrive/Loras/playful_origins_v4/dataset_config.toml
📄 Updated training config: /content/drive/MyDrive/Loras/playful_origins_v4/training_config.toml

⭐ Starting trainer...

E0000 00:00:1761802054.515490   11838 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761802054.522912   11838 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1761802054.539855   11838 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1761802054.539887   11838 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more tha

In [2]:
!pip install -U numpy==2.0.2 transformers==4.57.1 accelerate==1.11.0 diffusers==0.35.2 huggingface-hub==0.36.0 opencv-python==4.12.0.88 einops==0.8.1


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.2/19.2 MB 99.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 108.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.8/375.8 kB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 127.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 54.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.4/64.4 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.8/485.8 kB 49.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 64.1 MB/s eta 0:00:00
  Attempting uninstall: safetensors
    Found existing installation: safetensors 0.4.2
    Uninstalling safetensors-0.4.2:
   

In [1]:
from google.colab import drive
drive.mount('/content/drive')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## *️⃣ Extras

You can run these before starting the training.

In [ ]:
#@markdown ### 🔮 Optimizer
#@markdown If you run this cell you will change the optimizer used for training. The default will be `AdamW8bit` otherwise, which is recommended.<p>
#@markdown * Dadapt and Prodigy manage learning rate automatically and are very good for small datasets. You can use it without changing anything else here.<p>
#@markdown For Dadapt and Prodigy, the following values will be overriden if the box is checked:<p>
#@markdown `learning_rate=0.5`, `network_alpha=network_dim`, `lr_scheduler="constant_with_warmup"`, `lr_warmup_ratio=0.05`<p>
#@markdown For Dadapt and Prodigy, if `optimizer_args` is left empty the default will be `decouple=True, weight_decay=0.01, betas=[0.9,0.999]`<p>
#@markdown And additionally for Prodigy: `d_coef=2, use_bias_correction=True, safeguard_warmup=True`<p>
optimizer = "Prodigy" #@param ["AdamW8bit", "Prodigy", "DAdaptation", "DadaptAdam", "DadaptLion", "AdamW", "Lion", "SGDNesterov", "SGDNesterov8bit", "AdaFactor"]
optimizer_args = "" #@param {type:"string"}
splitter = ", " if ", " in optimizer_args else ","
optimizer_args = [a.strip() for a in optimizer_args.split(splitter) if a]
override_values_for_dadapt_and_prodigy = True #@param {type:"boolean"}

### 📚 Multiple folders in dataset
Below is a template allowing you to define multiple folders in your dataset. You must include the location of each folder and you can set different number of repeats for each one. To add more folders simply copy and paste the sections starting with `[[datasets.subsets]]`.

When enabling this, the number of repeats set in the main cell will be ignored, and the main folder set by the project name will also be ignored.

You can make one of them a regularization folder by adding `is_reg = true`  
You can also set different `keep_tokens`, `flip_aug`, etc.

In [3]:
custom_dataset = """
[[datasets]]
resolution = 768
batch_size = 2

[[datasets.subsets]]
image_dir = "/content/drive/MyDrive/Loras/playful_origins_v4/dataset_v2/class1/PlayfulOrigins_00001"
num_repeats = 10
keep_tokens = 1
shuffle_caption = true
caption_extension = ".txt"
"""

In [ ]:
custom_dataset = None

In [ ]:
#@markdown ### 🤓 Other
#@markdown These are kept here for the few people that use them.

#@markdown Weighted captions is a new feature that allows you to use (parentheses) to give more weight to certain tags in your dataset, same as in your webui prompts. <p>
#@markdown Normal parentheses in your tags such as `(series names)` will need to be escaped like `\(series names\)`
weighted_captions = False #@param {type:"boolean"}

#markdown By enabling `adjust_tags`, you will let this colab modify your tags before running to automatically adjust to `weighted_captions` being on or off. <p>
#markdown Then, you may increase `activation_tag_weight` to improve how effective your activation tag is.
adjust_tags = False #param {type:"boolean"}
activation_tag_weight = "1.0" #param ["1.0","1.1","1.2"]
keep_tokens_weight = float(activation_tag_weight)

#@markdown Here you can write a path in your Google Drive to load an existing Lora file to continue training on.<p>
#@markdown **Warning:** It's not the same as one long training session. The epochs start from scratch, and it may have worse results.
continue_from_lora = "" #@param {type:"string"}
if continue_from_lora and not continue_from_lora.startswith("/content/drive/MyDrive"):
  import os
  continue_from_lora = os.path.join("/content/drive/MyDrive", continue_from_lora)


In [ ]:
#@markdown ### 📂 Unzip dataset
#@markdown It's much slower to upload individual files to your Drive, so you may want to upload a zip if you have your dataset in your computer.
zip = "/content/drive/MyDrive/my_dataset.zip" #@param {type:"string"}
extract_to = "/content/drive/MyDrive/Loras/example/dataset" #@param {type:"string"}

import os, zipfile

if not os.path.exists('/content/drive'):
  from google.colab import drive
  print("📂 Connecting to Google Drive...")
  drive.mount('/content/drive')

os.makedirs(extract_to, exist_ok=True)

with zipfile.ZipFile(zip, 'r') as f:
  f.extractall(extract_to)

print("✅ Done")


In [1]:
from google.colab import drive
import os

# Mount Drive
drive.mount('/content/drive', force_remount=True)

# Verify dataset and model paths
paths_to_check = [
    "/content/drive/MyDrive/Loras/playful_origins_v4/dataset_v2/class1/PlayfulOrigins_00001",
    "/content/drive/MyDrive/Loras/playful_origins_v4/sd-nsfw-v1-5-realism.ckpt",
    "/content/drive/MyDrive/Loras/playful_origins_v4/output_lora"
]

for p in paths_to_check:
    print(f"Checking: {p}")
    if os.path.exists(p):
        print("✅ Path found\n")
    else:
        print("❌ Missing — check spelling or mount issues\n")


Mounted at /content/drive
Checking: /content/drive/MyDrive/Loras/playful_origins_v4/dataset_v2/class1/PlayfulOrigins_00001
✅ Path found

Checking: /content/drive/MyDrive/Loras/playful_origins_v4/sd-nsfw-v1-5-realism.ckpt
✅ Path found

Checking: /content/drive/MyDrive/Loras/playful_origins_v4/output_lora
✅ Path found



In [2]:
#@markdown ### 🔢 Count datasets
#@markdown Google Drive makes it impossible to count the files in a folder, so this will show you the file counts in all folders and subfolders.
folder = "/content/drive/MyDrive/Loras/playful_origins_v4/dataset_v2/class1/PlayfulOrigins_00001" #@param {type:"string"}

import os
from google.colab import drive

if not os.path.exists('/content/drive'):
    print("📂 Connecting to Google Drive...\n")
    drive.mount('/content/drive')

tree = {}
exclude = ("_logs", "/output")
for i, (root, dirs, files) in enumerate(os.walk(folder, topdown=True)):
  dirs[:] = [d for d in dirs if all(ex not in d for ex in exclude)]
  images = len([f for f in files if f.lower().endswith((".png", ".jpg", ".jpeg"))])
  captions = len([f for f in files if f.lower().endswith(".txt")])
  others = len(files) - images - captions
  path = root[folder.rfind("/")+1:]
  tree[path] = None if not images else f"{images:>4} images | {captions:>4} captions |"
  if tree[path] and others:
    tree[path] += f" {others:>4} other files"

pad = max(len(k) for k in tree)
print("\n".join(f"📁{k.ljust(pad)} | {v}" for k, v in tree.items() if v))


📁PlayfulOrigins_00001 |   30 images |   30 captions |


# 📈 Plot training results
You can do this after running the trainer. You don't need this unless you know what you're doing.  
The first cell below may fail to load all your logs. Keep trying the second cell until all data has loaded.

In [ ]:
%load_ext tensorboard
%tensorboard --logdir={log_folder}/

In [ ]:
from tensorboard import notebook
notebook.display(port=6006, height=800)

In [5]:
# --- ElevenSparks Angel Run Pre-Flight Check (v2) ---
from google.colab import drive
import os, glob

print("🔄 Mounting Google Drive...")
drive.mount('/content/drive', force_remount=True)

# --- Key Paths ---
dataset_path = "/content/drive/MyDrive/Loras/playful_origins_v4/dataset_v2/class1/PlayfulOrigins_00001"
model_path   = "/content/drive/MyDrive/Loras/playful_origins_v4/sd-nsfw-v1-5-realism.ckpt"
output_path  = "/content/drive/MyDrive/Loras/playful_origins_v4/output_lora"

# --- Verify Critical Paths ---
print("\n🔍 Verifying required paths...\n")
for p in [dataset_path, model_path, output_path]:
    print(f"Checking: {p}")
    print("✅ Exists\n" if os.path.exists(p) else "❌ MISSING — fix path before training!\n")

# --- Count Dataset Files ---
print("📊 Counting dataset contents...\n")

# include .jpg, .jpeg, and .png
img_files = (
    glob.glob(os.path.join(dataset_path, "*.jpg")) +
    glob.glob(os.path.join(dataset_path, "*.jpeg")) +
    glob.glob(os.path.join(dataset_path, "*.png"))
)
txt_files = glob.glob(os.path.join(dataset_path, "*.txt"))

print(f"🖼️ Images found: {len(img_files)}")
print(f"📝 Captions found: {len(txt_files)}")

if len(img_files) == len(txt_files) and len(img_files) > 0:
    print("\n✅ Dataset verified — matching image/caption pairs found.")
    print("✅ Base model and output directories accessible.")
    print("\n🚀 Ready to launch the 2222-step Angel Run. Proceed to the START TRAINING cell.")
else:
    # Show mismatches if any
    img_names = {os.path.splitext(os.path.basename(f))[0] for f in img_files}
    txt_names = {os.path.splitext(os.path.basename(f))[0] for f in txt_files}
    missing_imgs = sorted(txt_names - img_names)
    missing_txts = sorted(img_names - txt_names)

    print("\n⚠️ WARNING: Mismatch or missing files detected.")
    print(f"Missing image files for captions: {missing_imgs}")
    print(f"Missing caption files for images: {missing_txts}")
    print("\nFix mismatched names or add missing pairs before training.")


🔄 Mounting Google Drive...
Mounted at /content/drive

🔍 Verifying required paths...

Checking: /content/drive/MyDrive/Loras/playful_origins_v4/dataset_v2/class1/PlayfulOrigins_00001
✅ Exists

Checking: /content/drive/MyDrive/Loras/playful_origins_v4/sd-nsfw-v1-5-realism.ckpt
✅ Exists

Checking: /content/drive/MyDrive/Loras/playful_origins_v4/output_lora
✅ Exists

📊 Counting dataset contents...

🖼️ Images found: 30
📝 Captions found: 30

✅ Dataset verified — matching image/caption pairs found.
✅ Base model and output directories accessible.

🚀 Ready to launch the 2222-step Angel Run. Proceed to the START TRAINING cell.
